#### Data Preprocessing

In [1]:
from boxsers.preprocessing import savgol_smoothing, cosmic_filter

def preprocessing_method(x):
    # 1) Applies a median filter to remove cosmic rays from the spectrum(s).
    x = cosmic_filter(x, ks=3)
    # 2) Smoothes the spectra
    x = savgol_smoothing(x, 7, p=3, degree=0)
    return x

2024-07-30 21:29:19.107809: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 21:29:19.107885: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 21:29:19.109054: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


#### Loading Dataset

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from boxsers.misc_tools import data_split

df = pd.read_hdf('./data/Bile_acids_27_07_2020.h5', key='df')
spectrum = df.iloc[:, 1:].to_numpy()
spectrum = preprocessing_method(spectrum)  
spectrum_label = df.loc[:, 'Classes']

label_encoder = LabelEncoder()
numeric_labels = label_encoder.fit_transform(spectrum_label)
(sp_train, sp_test, lab_train, lab_test) = data_split(spectrum, numeric_labels, b_size=0.15, rdm_ste=1, print_report=False)

##### Model Setting

In [3]:
from model.Variant_LeNet_without_linear import Variant_LeNet_without_linear
from tqdm import tqdm
import torch
import pandas as pd
from torch.autograd import Variable
from functools import partial
from deep_SLDA import slda_loss, SLDA
from imblearn.metrics import specificity_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import torch.optim as optim
import seaborn as sns
from sklearn.metrics import confusion_matrix
import math
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    auc,
    roc_curve,
)
from plot import plot_bile_acids_ROC_curve, plot_bile_acids_heatmap, plot_loss_metrics, plot_metrics

n_classes = 6
batch_size = 4000

train_avg_accuracy = []
val_avg_accuracy = []
avg_accuracy = []
avg_roc = []
C = np.zeros((6, 6))

In [4]:
class Solver:
    def __init__(
        self,
        dataloaders,
        model,
        model_path,
        device,
        n_classes,
    ):
        self.dataloaders = dataloaders
        self.device = device
        self.net = model
        self.net = self.net.to(self.device)

        self.criterion = partial(
            slda_loss,
            n_classes=n_classes,
        )

        self.optimizer = optim.Adam(self.net.parameters(), lr=1e-4, betas=(0.5, 0.999))
        self.model_path = model_path
        self.n_classes = n_classes
        self.slda_layer = SLDA(self.n_classes)

    def iterate(self, epoch, phase, scheduler=None):
        if phase == "train":
            self.net.train()
        else:
            self.net.eval()

        dataloader = self.dataloaders[phase]
        total_loss = 0
        correct = 0
        total = 0
        loss_total = 0

        # if phase == "train":
        #     self.optimizer.zero_grad()

        for batch_idx, (inputs, targets) in enumerate(dataloader):
            inputs = Variable(inputs).to(self.device)
            targets= Variable(targets).to(self.device)

            feas = self.net(inputs)

            if phase == "train":
                dirs, range_eigenvalue, null_eigenvalue = self.slda_layer.fit(feas, targets, phase)
                Z = torch.matmul(feas, dirs.T)
                self.clf = LinearDiscriminantAnalysis()
                self.clf.fit(Z.detach().data.cpu().numpy(),targets.cpu().numpy())
                outputs = self.clf.predict(Z.detach().data.cpu().numpy())
                outputs = torch.from_numpy(outputs).to(self.device)
                loss = self.criterion(range_eigenvalue, null_eigenvalue)
                self.dirs = dirs
            else:
                range_eigenvalue, null_eigenvalue = self.slda_layer.fit(feas, targets, phase)
                Z = torch.matmul(feas, self.dirs.T)
                outputs = self.clf.predict(Z.detach().data.cpu().numpy())
                outputs = torch.from_numpy(outputs).to(self.device)
                loss = self.criterion(range_eigenvalue, null_eigenvalue)
                
            if phase == "train":
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

            total_loss += loss.item()
            total += targets.size(0)
            loss_total += 1
            correct += outputs.eq(targets).cpu().sum().item()    
        
        avg_loss = total_loss / loss_total
        total_acc = correct / total

        return avg_loss, total_acc

    def train(self, epochs):

        best_acc = 0

        useful_stuff = {
            "training_loss": [],
            "validation_loss": [],
            "train_metrics": [],
            "validation_metrics": [],
        }

        # lambda1 = lambda epoch: 0.9 ** (epoch // 20) if epoch >= 20 else 1.0
        # self.scheduler = torch.optim.lr_scheduler.LambdaLR(self.optimizer, lr_lambda=lambda1)

        for epoch in tqdm(range(epochs)):
            
            train_loss, train_acc = self.iterate(epoch, "train")
            useful_stuff["training_loss"].append(train_loss)
            useful_stuff["train_metrics"].append(train_acc)

            # self.optimizer.step()
            # self.scheduler.step()

            with torch.no_grad():
                val_loss, val_acc = self.iterate(epoch, "val")
                useful_stuff["validation_loss"].append(val_loss)
                useful_stuff["validation_metrics"].append(val_acc)

            
            if val_acc > best_acc or epoch == 0:
                best_acc = val_acc
                checkpoint = {
                    "epoch": epoch,
                    "val_loss": val_loss,
                    "dirs": self.dirs,
                    "clf": self.clf,
                    "state_dict": self.net.state_dict(),
                }
                torch.save(checkpoint, self.model_path)
            
        return train_acc, best_acc, useful_stuff

    def test_iterate(self, epoch, phase):
        self.net.eval()
        dataloader = self.dataloaders[phase]
        y_pred = []
        y_true = []
        y_pred_prob = []
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(dataloader):
                inputs, targets = Variable(inputs.cuda()), Variable(
                    targets.cuda()
                )
                feas = self.net(inputs)
                Z = torch.matmul(feas, self.dirs.T)
                outputs = self.clf.predict(Z.detach().data.cpu().numpy())
                outputs = torch.from_numpy(outputs).to(self.device)
                outputs_prob = self.clf.predict_proba(Z.detach().data.cpu().numpy())
                outputs_prob = torch.from_numpy(outputs_prob).to(self.device)

                y_pred.append(outputs.cpu().numpy().ravel())
                y_true.append(targets.cpu().numpy())
                y_pred_prob.append(outputs_prob.cpu().numpy())
            pass

        y_pred_prob = np.concatenate(y_pred_prob)
        y_pred = np.concatenate(y_pred)
        y_true = np.concatenate(y_true)

        return (
            np.array(y_pred).flatten(),
            np.array(y_true).flatten(),
            np.array(y_pred_prob).reshape(720, 6),
        )

    def test(self):
        checkpoint = torch.load(self.model_path)
        epoch = checkpoint["epoch"]
        val_loss = checkpoint["val_loss"]
        self.dirs = checkpoint["dirs"]
        self.clf = checkpoint["clf"]

        self.net.load_state_dict(checkpoint["state_dict"])
        print("load model at epoch {}, with val loss: {:.3f}".format(epoch, val_loss))
        y_pred, y_true, y_pred_prob = self.test_iterate(epoch, "test")
        print("total", accuracy_score(y_true, y_pred))
        for i in range(self.n_classes):
            idx = y_true == i
            print("class", i, accuracy_score(y_true[idx], y_pred[idx]))

        return (
            confusion_matrix(y_true, y_pred),
            y_true,
            y_pred,
            accuracy_score(y_true, y_pred),
            y_pred_prob,
        )

In [5]:
from sklearn.model_selection import StratifiedKFold
from datasets_spectrum import spectral_dataloader

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_index = 1

for train_idx, valid_idx in kfold.split(sp_train, lab_train):

    print("fold:", fold_index)
    x_train, y_train = sp_train[train_idx], lab_train[train_idx]
    x_valid, y_valid = sp_train[valid_idx], lab_train[valid_idx]
    
    print("train size: ", len(x_train))
    print("validation size: ", len(x_valid))
    print("test size: ", len(sp_test))

    dl_tr = spectral_dataloader(
        x_train, y_train, idxs=None, batch_size=batch_size, shuffle=True
    )
    dl_val = spectral_dataloader(
        x_valid, y_valid, idxs=None, batch_size=batch_size, shuffle=False
    )
    dl_test = spectral_dataloader(sp_test, lab_test, batch_size=batch_size, shuffle=False)

    values, counts = np.unique(np.asarray(lab_test), return_counts=True)

    dataloaders = {"train": dl_tr, "val": dl_val, "test": dl_test}
    model = Variant_LeNet_without_linear(in_channels=1)

    model_path = f"best_bile_acids_variant_lenet_model_{fold_index}.pt"
    solver = Solver(
        dataloaders, model, model_path, "cuda", n_classes
    )
    
    train_accuracy, val_accuracy, useful_stuff = solver.train(500)
    C, y_true, y_pred, test_accuracy, y_pred_prob = solver.test()
    train_avg_accuracy.append(train_accuracy)
    val_avg_accuracy.append(val_accuracy)
    avg_accuracy.append(np.round(test_accuracy,4))
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(np.unique(y_true).shape[0]):
        fpr[i], tpr[i], _ = roc_curve(lab_test == i, y_pred_prob[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    values = [
        v
        for v in roc_auc.values()
        if isinstance(v, (int, float)) and not math.isnan(v)
    ]
    if values:
        auc_score = sum(values) / len(values)
    avg_roc.append(auc_score)

    cm = confusion_matrix(y_true, y_pred, labels=[0,1,2,3,4,5])
    sns.set_context("talk", rc={"font": "Helvetica", "font.size": 12})
    label = ["Blank", "CA", "DCA", "GCDCA", "LCA", "TCDCA"]
    cm = 100 * cm / cm.sum(axis=1)[:,np.newaxis]

    accuracy = accuracy_score(y_true, y_pred)
    sensitivity = recall_score(y_true, y_pred, average="micro", zero_division=0)
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    f1 = f1_score(y_true, y_pred, average="micro", zero_division=0)

    df = pd.DataFrame(
        {
            "Accuracy": [np.round(accuracy_score(y_true, y_pred), 4)],
            "Recall": [
                recall_score(y_true, y_pred, average=None, zero_division=0).round(4)
            ],
            "Specificity": [specificity_score(y_true, y_pred, average=None).round(4)],
            "Precision": [
                precision_score(y_true, y_pred, average=None, zero_division=0).round(4)
            ],
            "F1 Score": [
                f1_score(y_true, y_pred, average=None, zero_division=0).round(4)
            ],
        }
    )
    print(df.transpose())

    plot_bile_acids_ROC_curve("variant_lenet", y_true, lab_test, y_pred_prob, fold_index=fold_index)
    plot_bile_acids_heatmap("variant_lenet", cm, fold_index=fold_index)
    plot_metrics(training_results=useful_stuff, fold_index=fold_index, fold_name="variant_lenet")
    plot_loss_metrics(training_results=useful_stuff, fold_index=fold_index, fold_name="variant_lenet")

    fold_index += 1

fold: 1
train size:  3264
validation size:  816
test size:  720


  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 1/500 [00:03<25:38,  3.08s/it]/home/luhung/.venv/torch2/lib/python3.10/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:1600: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  warnings.warn("k >= N for N * N square matrix. "
  0%|          | 2/500 [00:05<21:31,  2.59s/it]/home/luhung/.venv/torch2/lib/python3.10/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:1600: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  warnings.warn("k >= N for N * N square matrix. "
  1%|          | 3/500 [00:07<20:08,  2.43s/it]/home/luhung/.venv/torch2/lib/python3.10/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:1600: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  warnings.warn("k >= N for N * N square matrix. "
  1%|          | 4/500 [00:09<19:29,  2.36s/it]/home/luhung/.venv/torch2/lib/python3.10/site-packages/scipy/spar

load model at epoch 446, with val loss: 4.096


total 0.975
class 0 0.959349593495935
class 1 0.9401709401709402
class 2 0.9739130434782609
class 3 0.9910714285714286
class 4 0.9849624060150376
class 5 1.0
                                                         0
Accuracy                                             0.975
Recall        [0.9593, 0.9402, 0.9739, 0.9911, 0.985, 1.0]
Specificity  [0.9983, 0.9934, 0.9934, 0.9868, 0.9983, 1.0]
Precision    [0.9916, 0.9649, 0.9655, 0.9328, 0.9924, 1.0]
F1 Score      [0.9752, 0.9524, 0.9697, 0.961, 0.9887, 1.0]
fold: 2
train size:  3264
validation size:  816
test size:  720


...

load model at epoch 227, with val loss: 4.063


total 0.975
class 0 0.975609756097561
class 1 0.9743589743589743
class 2 0.9826086956521739
class 3 0.9553571428571429
class 4 0.9624060150375939
class 5 1.0
                                                         0
Accuracy                                             0.975
Recall       [0.9756, 0.9744, 0.9826, 0.9554, 0.9624, 1.0]
Specificity  [0.9966, 0.9934, 0.9884, 0.9934, 0.9983, 1.0]
Precision     [0.9836, 0.9661, 0.9417, 0.964, 0.9922, 1.0]
F1 Score     [0.9796, 0.9702, 0.9617, 0.9596, 0.9771, 1.0]
fold: 5
train size:  3264
validation size:  816
test size:  720


100%|██████████| 500/500 [19:14<00:00,  2.31s/it]

load model at epoch 357, with val loss: 3.963


total 0.9763888888888889
class 0 0.975609756097561
class 1 0.9572649572649573
class 2 0.9652173913043478
class 3 0.9910714285714286
class 4 0.9699248120300752
class 5 1.0
                                                         0
Accuracy                                            0.9764
Recall       [0.9756, 0.9573, 0.9652, 0.9911, 0.9699, 1.0]
Specificity   [0.9966, 0.995, 0.9934, 0.9901, 0.9966, 1.0]
Precision    [0.9836, 0.9739, 0.9652, 0.9487, 0.9847, 1.0]
F1 Score     [0.9796, 0.9655, 0.9652, 0.9694, 0.9773, 1.0]


In [6]:
print(avg_accuracy)
print("train mean:", round(np.mean(train_avg_accuracy),4))
print("train std:", round(np.std(train_avg_accuracy),4))

print("val mean:", round(np.mean(val_avg_accuracy),4))
print("val std:", round(np.std(val_avg_accuracy),4))

print("test mean:", round(np.mean(avg_accuracy),4))
print("test std:", round(np.std(avg_accuracy),4))

print("auc mean:", round(np.mean(avg_roc),4))
print("auc std:", round(np.std(avg_roc),4))

[0.975, 0.9806, 0.975, 0.975, 0.9764]
train mean: 0.9971
train std: 0.0014
val mean: 0.973
val std: 0.0046
test mean: 0.9764
test std: 0.0022
auc mean: 0.9991
auc std: 0.0003


: 